In [1]:
import subprocess
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import List

In [2]:
# Change working directory to the PathTracer
os.chdir("C:/Users/mathias.gammelmark/Documents/repo/LightSourceImportanceSampling/PathTracer")

In [3]:
def run(args:List[str]):
    process = subprocess.run(["../bin/Release-windows-x86_64/PathTracer.exe"] + args, stdout=None)

def arg_num_samples(n:int):
    return ["-n", str(n)]

def arg_load_obj(filepath:str):
    return ["-obj", filepath]

def arg_scene(files:List[str]):
    res = []
    for file in files:
        res = res + arg_load_obj(file)
    return res

def arg_cam_pos(x:float, y:float, z:float):
    return ["-pos", str(x), str(y), str(z)]

def arg_cam_rot(x:float, y:float, z:float):
    return ["-rot", str(x), str(y), str(z)]

def arg_output(filepath:str):
    return ["-out", filepath]

In [4]:
figure_dpi = 100

def load_image_from_csv(filepath:str):
    data = pd.read_csv(filepath).to_numpy()
    #print(np.shape(data))
    img = np.zeros((512,512,3), dtype=np.float)

    for y in range(0,511):
        for x in range(0, 511):
            pixel = data[(511-y)*512 + x][:]
            img[y,x] = pixel

    return img

def exposure(img, exposure):
    # reinhard tone mapping
    return img / (img+exposure)

def gamma(img, gamma):
    return np.power(img, 1.0 / gamma)

def color_correct(img):
    return gamma(exposure(img, 1.0), 2.2)

def show_img(img):
    plt.figure(dpi=figure_dpi)
    plt.imshow(color_correct(img))
    plt.show()

In [5]:
models = ["../Assets/Models/Helix.obj", "../Assets/Models/CornellBox.obj"]

args_scene = arg_scene(models)

print("Running Pathtracer")
run(args_scene + arg_num_samples(1))
print("Loading Image")
resA = load_image_from_csv("../Test/Test.csv")
show_img(resA)



Running Pathtracer


NameError: name 'NONE' is not defined

In [ ]:
print("Running Pathtracer")
run(args_scene + arg_num_samples(1000))
print("Loading Image")
resB = load_image_from_csv("../Test/Test.csv")
show_img(resB)